<a href="https://colab.research.google.com/github/JustAlex5/deep_learning/blob/main/VGG_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models,Input
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,Dropout
import numpy as np
from tensorflow.keras.layers import SimpleRNN, LSTM ,RepeatVector,Reshape# <--- recurrent layers
from tensorflow.keras.utils import to_categorical
import re
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator, array_to_img
from PIL import Image
import datetime
from keras.preprocessing.image import load_img
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Dense, Dropout, LSTM, Embedding
from keras.layers.merge import add

# Load the TensorBoard notebook extension
%load_ext tensorboard

from keras.applications.vgg16 import VGG16
print(tf.__version__)
PATH="/content/drive/MyDrive/Fina_Prj/"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
2.7.0


In [2]:
!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_Dataset.zip

!wget -q https://github.com/jbrownlee/Datasets/releases/download/Flickr8k/Flickr8k_text.zip

!unzip -qq Flickr8k_Dataset.zip

!unzip -qq Flickr8k_text.zip

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
image_path="/content/drive/MyDrive/Fina_Prj/Flicker8k_Dataset/"
txt_path="/content/drive/MyDrive/Fina_Prj/Flickr8k_text/"

In [5]:
dict={}
data = open(txt_path+'Flickr8k.lemma.token.txt', 'r')

lines= data.readlines()
max_seq=0
# creat dictionary of images (train and test)
# each img contain all the descriptions 
# convert to lower case and delete new line char
for line in lines:
    a,b=line.split("\t",1)
    a=a.split("#")
    b=re.sub(r'[^a-zA-Z0-9 ]',r'',b.lower())
    b=b.split(" ")
    b.insert(0,'<start>')
    b.insert(len(b)-1,'<stop>')
    b=b[:-1]
    b=" ".join(b)
    if a[0]  in dict.keys():       
        dict[a[0]].append(b)
    else: 
        dict.update({a[0]:[b]})

train_dict={}

# open the train images list
data_train = open(txt_path+'Flickr_8k.trainImages.txt', 'r')
lines= data_train.readlines()
# copy the descriptions of only train images to new dictionary
for line in lines:
    line=line.replace('\n','')
    if line in dict.keys():
        train_dict[line]=dict[line]

words={}
new_word={}
train_dict_new={}
# count all word in the end there dictionary with all words in the train and times that repeats 
for key_item in train_dict:
    value_item= train_dict[key_item]
    tmp_item=" ".join(value_item)
    tmp_item=tmp_item.split(" ")
    for item in tmp_item:
        if len(item) ==1:
            continue
        if item in words.keys():
            words.update({item:words.get(item)+1})
        else:
            words.update({item:1})
for key_item in words:
    value_item= words[key_item]
    if value_item >5:
        # copy all words to new dictionary 
        new_word[key_item]=words[key_item]

# delete all words that apears less than 5 times 
for key_item in train_dict:
    value_item=train_dict[key_item]
    tmp_string=",".join(value_item)
    tmp_string=tmp_string.split(",")
    res=[]
    for item in tmp_string:
        a=[]
        items=item.split(" ")
        if len(items)>max_seq:
            max_seq=len(items)
        for item in items:
            if item in new_word.keys():
                a.append(item)
        res.append(a)
        train_dict_new.update({key_item:res})   
final_dict={}
idx_word={}
idx_word.update({0:" "})

for x,y in enumerate(new_word):
    final_dict.update({y:x+1})
    idx_word.update({x+1:y})


X = np.zeros((len(train_dict_new),len(value_item),  max_seq), dtype='int')
Y = np.zeros((len(train_dict_new),len(value_item),  max_seq), dtype='int')

images=[]
X2=[]
Y_train=[]

for j,seqs in enumerate(train_dict_new.items()):
    for   i, seq in enumerate(seqs[1]):
        for t, word in enumerate(seq):
            if t==0:
                X[j,i,t]=final_dict[word]
                continue
            if word=='<stop>':
                Y[j,i,t-1]=final_dict[word]
                break
            X[j,i,t]=final_dict[word]
            Y[j,i,t-1]=final_dict[word]
        X2.append(X[j,i,:])
        Y_train.append(Y[j,i,:])
        images.append(seqs[0])

In [76]:
X[0,0,:]

array([ 1,  2,  3,  4,  5,  6,  7,  3,  8,  9, 10,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0])

In [77]:
X2.shape

(30000, 39)

In [6]:
# extract features from each photo in the directory
def extract_features( image_keys):
    # load the model
    model = VGG16()
    
    # re-structure the model
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    
    # summarize
    print(model.summary())
    
    # extract features from each photo
    features ={}
    
    for seqs in image_keys:
        
        # load an image from file
        filename = f'Flicker8k_Dataset/{seqs}'
        
        # load the image and convert it into target size of 224*224
        image = load_img(filename, target_size=(224, 224))
        
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        
        # prepare the image for the VGG model
        image = preprocess_input(image)
        
        # get features
        feature = model.predict(image, verbose=0)
        
        # get image id
        
        
        # store feature
        features.update({seqs:feature}) 
        
#         print('>%s' % name)
        

    return features

In [7]:
train_validate_features = extract_features(list(train_dict_new.keys()))


553476096/553467096 [==============================] - 5s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                            

In [61]:
X1=np.array(X1)

In [21]:
X1=[]
for _,mtrix in enumerate(train_validate_features.items()):
    X1.append(mtrix[1][0])

In [59]:
X1=[]
for _,mtrix in enumerate(train_validate_features.items()):
  for i in range (5):
    # import pdb; pdb.set_trace()
    X1.append(mtrix[1][0])

In [62]:
X1.shape

(30000, 4096)

In [30]:
X2=np.array(X2)

In [32]:
Y_train=np.array(Y_train)

In [71]:
inputs1= Input(shape=(4096,))
fe1=Dropout(0.5)(inputs1)
fe2=Dense(256,activation='relu')(fe1)

inputs2= Input(shape=(max_seq))
se1=Embedding(len(idx_word),256, mask_zero=True)(inputs2)
se2=Dropout(0.5)(se1)
se3=LSTM(256)(se2)

decoder1=add([fe2,se3])
decoder2=Dense(256,activation='relu')(decoder1)
outputs=Dense(max_seq,activation='softmax')(decoder2)

model=Model(inputs=[inputs1,inputs2], outputs=outputs)



In [72]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)

In [73]:
Y_train.shape

(30000, 39)

In [69]:
log_dir = "/content/drive/MyDrive/Fina_Prj/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [74]:
history=model.fit([X1,X2],Y_train,batch_size=64,epochs=10,verbose=1,callbacks=[tensorboard_callback])

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/10
469/469 [==============================] - 11s 11ms/step - loss: 253037792.0000 - accuracy: 0.1284
Epoch 2/10
469/469 [==============================] - 5s 10ms/step - loss: 2080877952.0000 - accuracy: 0.1286
Epoch 3/10
469/469 [==============================] - 5s 10ms/step - loss: 6883723264.0000 - accuracy: 0.1286
Epoch 4/10
469/469 [==============================] - 5s 10ms/step - loss: 15800643584.0000 - accuracy: 0.1286
Epoch 5/10
469/469 [==============================] - 5s 10ms/step - loss: 29765738496.0000 - accuracy: 0.1286
Epoch 6/10
469/469 [==============================] - 5s 10ms/step - loss: 51017019392.0000 - accuracy: 0.1286
Epoch 7/10
469/469 [==============================] - 5s 10ms/step - loss: 78888640512.0000 - accuracy: 0.1286
Epoch 8/10
469/469 [==============================] - 5s 10ms/step - loss: 117351907328.0000 - accuracy: 0.1286
Epoch 9/10
469/469 [==============================] - 5s 10ms/step - loss: 166493798400.0000 - accuracy: 0.1286
Ep

In [36]:
X.shape

(6000, 5, 39)